In [1]:
try:
  import google.colab
  from google.colab import files
  IN_COLAB = True
except:
  IN_COLAB = False

In [2]:
!pip install alive_progress
import pandas as pd
import math
import io
from datetime import datetime, time
from alive_progress import alive_bar
import os


# Transcript Mapping

### Set File Names and Variables

Set file names here, the upload boxes will appear under each section of the Read Data section. Please make sure that the real filenames match the file names entered here. 

In [3]:
# Controls if a sample will be used
# Set to [RANDOM, HEAD, TAIL, NONE]
FILTER = "RANDOM"
SIZE = 1000

# Training Records from SumTotal
filename = "2020-2022 Training Records.xlsx"

# Coursename Mapping Setup
USE_COURSE_MAP = True
CMAP_ORIG = "OLD_CNAME"
CMAP_NEW = "NEW_CNAME"
VERS_OLD = "OLD_VERS"
VERS_NEW = "NEW_VERS"
MAP_PATH = "map.csv"

# Saba Courses
COURSE_PATH = "saba_courses.csv"

# Saba People
PEOPLE_PATH = "saba_people.csv"

### Other Global Variables

In [4]:
USERNAME = "Employee Number"
COURSEID = "OFFERING"
COURSEVERSION = "COURSE_VERSION"
COMPDATE = "Completion Date"
STATUS = "Completion Status"
COURSETITLE = "Activity Name"
PASSFAIL = "Pass/Fail"
ATTSTATUS = "Attendance Status"
ACT_TYPE = "Activity Type"

FOLDER = "trans_output/"
OUTPUT_PRE = "transcript_out_"
ERRORS_PRE = "transcript_errors_"
MISSING_COURSE = "missing_course_"
MISSING_PEOPLE = "missing_people_"
FILE_EXT = ".csv"
TIME_SUFF = datetime.now().strftime('%Y_%m_%d-%H-%M')

## Read Data

#### Transcript List

In [5]:
### Note: if using Google Colab, this must be set to the filename of the 
# file you will load. This is a quirk of Colab. 
if IN_COLAB:
  print("Colab")
  uploaded = files.upload()
  source = pd.read_excel(io.BytesIO(uploaded.get(filename)))
else:
  print("Not Colab")
  source = pd.read_excel(filename)
#source = source[source[ACT_TYPE] == "Course"]

Not Colab


#### Mapping Data

In [6]:
def map_course(mapping_list, orig_df):
    new_df = orig_df
    for _, map_row in mapping_list.iterrows():
        old_course = map_row[CMAP_ORIG]
        new_course = map_row[CMAP_NEW]
        #new_df[[COURSEID]] = new_df[[COURSEID]].replace(old_course, new_course)
        new_df.replace(old_course, new_course, inplace=True)
        #print(old_course, new_course)
    return new_df

In [7]:
if USE_COURSE_MAP:
  if IN_COLAB:
    print("Colab")
    uploaded = files.upload()
    course_map = pd.read_csv(io.BytesIO((uploaded.get(MAP_PATH))))
  else:
    print("Not Colab")
    course_map = pd.read_csv(MAP_PATH)

Not Colab


#### Saba Courses

In [8]:
if IN_COLAB:
  print("Colab")
  uploaded = files.upload()
  courses = pd.read_csv(io.BytesIO((uploaded.get(COURSE_PATH))))
else:
  print("Not Colab")
  courses = pd.read_csv(COURSE_PATH)


Not Colab


#### Saba People

In [9]:
if IN_COLAB:
  print("Colab")
  uploaded = files.upload()
  people = pd.read_csv(io.BytesIO((uploaded.get(PEOPLE_PATH))))
else:
  print("Not Colab")
  people = pd.read_csv(PEOPLE_PATH)

Not Colab


## Begin Processing

In [10]:
# Generate Output Filename
def genFileName(prefix, suffix, extension):
    cwd = os.getcwd()+"/"
    return cwd+FOLDER+str(prefix)+str(suffix)+str(extension)

In [11]:
# Take Data Sample if Testing
if FILTER == "RANDOM":
    source = source.sample(SIZE)
elif FILTER == "HEAD":
    source = source.head(SIZE)
elif FILTER == "TAIL":
    source = source.tail(SIZE)


In [12]:
source.head(5)

,Employee Number,Employee Name,Waived Notes,Employee email,Job Title,Department Code,Department Name,MANAGER,Activity Type,OFFERING,Activity Name,COURSE_VERSION,Score,Attendance Status,Pass/Fail,Completion Status,Completion Date,Expiration Date,Note
70178,90413,"PROCEVIAT, John",NaN,john.proceviat@cnl.ca,Heavy Equip Operator,550.0,WL MAINTENANCE MANAGEMENT,"STRYMBISKI, Jeff",ILT Course,WL-306,Pre Job Brief,NaN,88.0,Attended,Pass,Completed,2021-05-11,NaT,NaN
30614,56463,"LUCKOVITCH, Michael",NaN,michael.luckov@cnl.ca,Maintenance Specialist,8599.0,WASTE SUPPORT SERVICES,"DELOUGHERY, Patrick Mark",Course,FIRE-1001-Online (Rev 4.4),Fire Prevention (4.4),4.4,90.0,Attended,Pass,Completed,2021-03-23,NaT,NaN
106341,92370,"PERRAULT-SANDERS, Ian",NaN,ian.perrault-sanders@cnl.ca,Utility Worker,8114.0,ZONE 4 - ROADS GROUNDS & TRANS,"SERNOSKIE, Peter",Versional,TD-1018-Online,CNL General Safety Orientation,NaN,100.0,Attended,NaN,Completed,2021-09-07,NaT,NaN
63204,60914,"LANG, Julien",NaN,Julien.Lang@cnl.ca,Research Scientist,3001.0,HYDROGEN TECHNOLOGIES,"RYLAND, Donald",Course,OSH-9045-Online (Rev 1),Cost Course - Hand Safety (1.0),1.0,100.0,Attended,Pass,Completed,2021-09-10,NaT,NaN
81059,90996,"LAMPMAN, Patrick",NaN,patrick.lampman@cnl.ca,Radiation Surveyor,8183.0,DECONTAMINATION CENTRE,"PRESCOTT, Adam",Course,ENG-1069-Online (REV 0.1),Fume Hood Safety (0.1),0.1,NaN,Attended,NaN,Completed,2021-12-13,NaT,NaN


### Generate Status Table (Optional)

In [13]:
#print(source.shape[1])
#source[["Completion Status", "Attendance Status", "Pass/Fail"]].groupby(["Pass/Fail","Completion Status", "Attendance Status"], dropna=False, as_index=False).size()

### Update Mapped Course Names

In [14]:
if USE_COURSE_MAP:
    source = map_course(course_map, source)
source.head(5)

,Employee Number,Employee Name,Waived Notes,Employee email,Job Title,Department Code,Department Name,MANAGER,Activity Type,OFFERING,Activity Name,COURSE_VERSION,Score,Attendance Status,Pass/Fail,Completion Status,Completion Date,Expiration Date,Note
70178,90413,"PROCEVIAT, John",NaN,john.proceviat@cnl.ca,Heavy Equip Operator,550.0,WL MAINTENANCE MANAGEMENT,"STRYMBISKI, Jeff",ILT Course,WL-306,Pre Job Brief,NaN,88.0,Attended,Pass,Completed,2021-05-11,NaT,NaN
30614,56463,"LUCKOVITCH, Michael",NaN,michael.luckov@cnl.ca,Maintenance Specialist,8599.0,WASTE SUPPORT SERVICES,"DELOUGHERY, Patrick Mark",Course,FIRE-1001-Online (Rev 4.4),Fire Prevention (4.4),4.4,90.0,Attended,Pass,Completed,2021-03-23,NaT,NaN
106341,92370,"PERRAULT-SANDERS, Ian",NaN,ian.perrault-sanders@cnl.ca,Utility Worker,8114.0,ZONE 4 - ROADS GROUNDS & TRANS,"SERNOSKIE, Peter",Versional,TD-1018-Online,CNL General Safety Orientation,NaN,100.0,Attended,NaN,Completed,2021-09-07,NaT,NaN
63204,60914,"LANG, Julien",NaN,Julien.Lang@cnl.ca,Research Scientist,3001.0,HYDROGEN TECHNOLOGIES,"RYLAND, Donald",Course,OSH-9045-Online (Rev 1),Cost Course - Hand Safety (1.0),1.0,100.0,Attended,Pass,Completed,2021-09-10,NaT,NaN
81059,90996,"LAMPMAN, Patrick",NaN,patrick.lampman@cnl.ca,Radiation Surveyor,8183.0,DECONTAMINATION CENTRE,"PRESCOTT, Adam",Course,ENG-1069-Online (REV 0.1),Fume Hood Safety (0.1),0.1,NaN,Attended,NaN,Completed,2021-12-13,NaT,NaN


### Read SABA Data

#### Course Data

In [15]:
# Process Course Data from Saba
course_id = courses[["Course ID", "Version"]]
ids_only = pd.Series(courses["Course ID"].unique())

courses.head()

,Title,Version,Course ID,Unnamed: 3
0,100035-IBM Cognos Analytics Advanced (Cost Cou...,1.0,IT-9050,NaN
1,100035-IBM Cognos Analytics Fundamentals (Cost...,1.0,IT-9049,NaN
2,10 Commandments of Training,1.0,TD-1048,NaN
3,12 GA Shotgun,1.0,SECU-3015,NaN
4,5S Methodology (Cost Course),1.0,PI-9006-ONLINE,NaN


#### Employee Data

In [16]:
# Process People Data from Saba
people_un = set(people["Username"].astype("int64", errors="ignore").unique())
#people.head()
len(people_un)

3869

### Data Processing Functions

In [17]:
def procStatus(pass_fail, comp_status, att_status):
    status = -1
    ## Fail
    if pass_fail == "Fail":
        status = 400
    elif (pass_fail == "Pass") & (comp_status == "Not Completed"):
        status = 400
    #elif (math.isnan(pass_fail)) & (math.isnan(comp_status)) & ((att_status == "No-show") | (att_status == "Replaced")):
    #    status = 400
    ## Pass
    elif (pass_fail == "Pass") & (comp_status != "Not Completed"):
        status = 200
    elif (pd.isnull(pass_fail)) & (comp_status == "Completed"):
        status = 200
        #print("no p/f, status is complete")
    elif ( (pd.isnull(pass_fail)) & (pd.isnull(comp_status)) & (att_status == "Attended") ):
        status = 200
        #print("no p/f, no status, attstatus is attended")
    else:
        status = 400
    return status

def checkPerson(unList, person):
    is_in = False
    if str(person) in unList:
        is_in = True
    return is_in

def checkCourse(courseList, courseID, version):
    is_in = False
    replace_version = version
    for _, row in courseList.iterrows():
        #print(courseID, version, row["Course ID"], row["Version"])
        if (courseID == row["Course ID"]):
            #print("Course Match", courseID, version, row["Course ID"], row["Version"])
            if pd.isnull(version):
                replace_version = row["Version"]
                #print("Insert Version", replace_version, version)
            #else:
                #print("Orig Version:", replace_version, version)
            is_in = True
            #print("Match:", courseID, replace_version, version)
    return is_in, replace_version

class TranscriptRow:
    def __init__(self, username, courseID, courseVersion, compDate, status, courseTitle, inCourse, inPerson):
        self.username = username
        self.courseID = courseID.upper()
        self.courseVersion = courseVersion
        self.compDate = compDate
        self.status = status
        self.courseTitle = courseTitle
        self.inCourse = inCourse
        self.inPerson = inPerson


In [18]:
# Process Data
source_used = source[[USERNAME, COURSEID, COURSEVERSION, COMPDATE, PASSFAIL, STATUS, ATTSTATUS, COURSETITLE]]

transcripts = []

with alive_bar(source_used.shape[0], force_tty=True) as bar_read:
    for index, trans in source_used.iterrows():
        username = trans[USERNAME]
        courseID = trans[COURSEID].upper()
        course_ver = trans[COURSEVERSION]
        comp_date = trans[COMPDATE].strftime('%Y-%m-%d')
        status = procStatus(trans[PASSFAIL], trans[STATUS], trans[ATTSTATUS])
        course_title = trans[COURSETITLE]
        in_course, rep_ver = checkCourse(course_id, courseID, course_ver)
        in_pers = checkPerson(people_un, username)

        #Update Version
        course_ver = rep_ver
        
        trans_entry = TranscriptRow(username, courseID, course_ver, comp_date, status, course_title, in_course, in_pers)
        transcripts.append(trans_entry)
        #print(courseID, type(courseID), courseID in ids_only.values)
        bar_read()
print("Transcripts Processed:", len(transcripts))

|████████████████████████████████████████| 1000/1000 [100%] in 56.4s (17.72/s)                                           ▇▅▃ 43/1000 [4%] in 3s (16.5/s, eta: 58s) █▆▄ 68/1000 [7%] in 4s (16.7/s, eta: 56s) (16.6/s, eta: 52s) ▂▂▄ 159/1000 [16%] in 10s (16.6/s, eta: 51s) 175/1000 [18%] in 11s (16.6/s, eta: 50s)  ▇▇▅ 196/1000 [20%] in 12s (16.6/s, eta: 48s) ▇▅▃ 200/1000 [20%] in 12s (16.6/s, eta: 48s)  ▄▂▂ 206/1000 [21%] in 12s (16.6/s, eta: 48s) ▃▁▃ 235/1000 [24%] in 14s (16.6/s, eta: 46s)  271/1000 [27%] in 16s (16.7/s, eta: 44s) ▂▂▄ 315/1000 [32%] in 19s (16.7/s, eta: 41s) ▆█▆ 327/1000 [33%] in 20s (16.6/s, eta: 41s)  ▆█▆ 349/1000 [35%] in 21s (16.5/s, eta: 40s) in 27s (16.6/s, eta: 33s) in 29s (16.6/s, eta: 32s) ▃▁▃ 520/1000 [52%] in 31s (16.6/s, eta: 29s)  ▇▇▅ 593/1000 [59%] in 35s (16.8/s, eta: 24s)  ▃▁▃ 700/1000 [70%] in 41s (17.1/s, eta: 18s)  705/1000 [70%] in 41s (17.2/s, eta: 17s)  ▄▆█ 712/1000 [71%] in 41s (17.2/s, eta: 17s)  ▅▇▇ 743/1000 [74%] in 43s (17.3/s, eta: 15s) ▆█▆ 773

### Print to File Outputs

In [19]:
# Process a Transcript Row
def write_row(df_output, trans, test=False, subset="ALL"):
    #df = df_output
    
    inCourse = trans.inCourse
    inPerson = trans.inPerson

    tmp_dict = {
        "LEARNER": trans.username,
        "COURSE": trans.courseID,
        "COURSE_VERSION": trans.courseVersion,
        "COMPLETION_DATE": trans.compDate,
        "COMPLETION_STATUS": trans.status,
        "CREATE_ADHOC_COURSE": "FALSE",
        "COURSE_TITLE": trans.courseTitle
    }
    if test == True:
        tmp_dict = {
            "LEARNER": trans.username,
            "COURSE": trans.courseID,
            "COURSE_VERSION": trans.courseVersion,
            "COMPLETION_DATE": trans.compDate,
            "COMPLETION_STATUS": trans.status,
            "CREATE_ADHOC_COURSE": "FALSE",
            "COURSE_TITLE": trans.courseTitle,
            "IN_COURSE": trans.inCourse,
            "IN_PERSON": trans.inPerson
        }
    if subset == "ALL":
        df_output = df_output.append(tmp_dict, ignore_index=True)
    elif subset == "CLEAN":
        if ( (inCourse == 1) & (inPerson == 1) ):
            df_output = df_output.append(tmp_dict, ignore_index=True)
    elif subset == "MISSING":
        if ( (inCourse == 0) | (inPerson == 0) ):
            tmp_dict["COURSE_GOOD"] = bool(inCourse)
            tmp_dict["PERSON_GOOD"] = bool(inPerson)
            df_output = df_output.append(tmp_dict, ignore_index=True)
    
    return df_output
    

In [20]:
# Output Header Values
out_header = [
    "LEARNER",
    "COURSE",
    "COURSE_VERSION",
    "COMPLETION_DATE",
    "COMPLETION_STATUS",
    "CREATE_ADHOC_COURSE",
    "COURSE_TITLE"
]

### Generate Saba Import Data Table and Write to File

In [21]:
out = pd.DataFrame(columns = out_header)

with alive_bar(len(transcripts), force_tty=True) as bar:
    for ts in transcripts:
        out = write_row(out, ts, False, "CLEAN")
        bar()
out["ID"] = out.index
out.sample(20).T

|████████████████████████████████████████| 1000/1000 [100%] in 1.7s (571.55/s)                                          


,292,2,251,320,52,8,528,530,225,202,379,298,328,414,68,128,557,285,101,445
LEARNER,57572,58525,61371,57812,55991,61234,54239,91484,92406,61448,95079,53186,20022,60321,10490,56219,57370,91748,50954,91944
COURSE,FIRE-1001-ONLINE,HR-1005,HR-1005,HR-1005,EMP-1036,FIN-1004-ONLINE,OSH-1004,HR-1005,FD-3019,TD-9043,RP-CONTRACTOR 7,EMP-1036,FIRE-3001,PMO-9011,WL-303,PROC-1019,FIRE-3050-CRL,RP-G1-035,OPS-9020-ONLINE,NMMS-1011-ONLINE
COURSE_VERSION,4.7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.6
COMPLETION_DATE,2021-02-09,2021-11-22,2021-11-29,2021-11-17,2022-02-25,2022-03-28,2022-01-26,2021-11-30,2022-01-27,2021-04-20,2022-04-11,2022-04-14,2021-10-27,2021-03-10,2021-01-18,2021-07-22,2021-06-10,2021-01-12,2021-04-10,2021-09-07
COMPLETION_STATUS,400,400,400,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200
CREATE_ADHOC_COURSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
COURSE_TITLE,Fire Prevention,Virtual Mandatory Harassment and Violence Prev...,Mandatory Harassment and Violence Prevention T...,Mandatory Harassment and Violence Prevention T...,Emergency Procedure Refresher,Financial Delegation Of Authority,Lock Out / Tag Out,Virtual Mandatory Harassment and Violence Prev...,FD&ER Integrated Work Control,Cost Course - Coaching Skills for Effective Le...,RWP Overview,Emergency Procedure Refresher (0.2),Fire Extinguisher - Practical,Cost Course - $495/person VIRTUAL Project Mana...,International Leadership Training,VIRTUAL CNL Contractor Management Overview,Fire Hose: Practical: Supply Hose Lays (Skills...,M10 C4 - Incident response principles,Process and Instrumentation Diagrams,Nuclear Criticality Safety Program Awareness
ID,292,2,251,320,52,8,528,530,225,202,379,298,328,414,68,128,557,285,101,445


In [22]:
if IN_COLAB:
    out.to_csv("out_transcript.csv", index=False)
else:
    out.to_csv(genFileName(OUTPUT_PRE, TIME_SUFF, FILE_EXT), index=False)

In [23]:
# OLD OUTPUT
#print(out.shape)
#out.to_csv("out_transcript.csv", index=False)
#out.to_csv(genFileName(OUTPUT_PRE, TIME_SUFF, FILE_EXT), index=False)

#### Write Failiures to File

In [24]:
out_err = pd.DataFrame(columns = out_header)

with alive_bar(len(transcripts), force_tty=True) as bar:
    for ts in transcripts:
        out_err = write_row(out_err, ts, False, "MISSING")
        bar()
out_err["ID"] = out_err.index
out_err.sample(20).T

|████████████████████████████████████████| 1000/1000 [100%] in 1.4s (736.94/s)                                          


,343,158,351,76,27,398,165,257,406,212,220,131,169,267,261,222,272,234,156,255
LEARNER,61022,CONT-588,90592,59792,CN85043,61512,90910,92247,52987,91381,92372,90670,91203,92436,92299,CONT-3104,92523,92098,327,91340
COURSE,OSH-9045-ONLINE (REV 1),RP-G4,OSH-1006-ONLINE (REV 1.1),EMP-1037-ONLINE (REV 0.1),RP-G3-FCO,FIRE-1001-ONLINE (REV 4.5),OSH-1004-ONLINE,SECU-1008-ONLINE (REV 3.9),LGL-1003-ONLINE (REV 3.3),WCS-1008,SECU-1006-ONLINE,TD-1018-ONLINE (REV 1.4),NMMS-1011-ONLINE (REV 3.5),NMMS-1011-ONLINE (REV 3.6),OSH-9045-ONLINE,CNL-1010,SECU-2008-ONLINE (REV 1.6),TD-9030-ONLINE,OSH-5001,SECU-2008-ONLINE (REV 1.0)
COURSE_VERSION,1.0,NaN,1.1,0.1,1.0,4.5,NaN,3.9,3.3,NaN,NaN,1.4,3.5,3.6,1.0,1.0,1.6,NaN,NaN,1.0
COMPLETION_DATE,2022-01-06,2022-02-14,2021-05-13,2022-01-26,2021-03-10,2021-12-09,2021-11-01,2021-06-25,2022-02-23,2022-02-02,2022-03-14,2021-04-05,2021-04-21,2021-10-05,2021-11-22,2022-01-10,2021-12-14,2021-03-23,2021-10-05,2021-03-08
COMPLETION_STATUS,200,200,200,200,200,200,200,200,200,400,200,200,200,200,200,200,200,200,200,200
CREATE_ADHOC_COURSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
COURSE_TITLE,Cost Course - Hand Safety (1.0),Group 4 - Radiation Protection Training (R5),Confined Space Exam (1.1),Emergency Procedure Refresher (0.1),Group 3 Practical Field Checkout,Fire Prevention (4.5),Lock Out Tag Out Exam,Security Awareness (3.9),Values and Ethics at CNL (3.3),ERM HBK Module 8: NDA Overview,Level 2 Security Clearance Awareness,CNL General Safety Orientation (1.4),Nuclear Criticality Safety Program Awareness (...,Nuclear Criticality Safety Program Awareness (...,Hand Safety,CNL COVID-19 Pandemic Response,Security Awareness Refresher (1.6),Harassment Prevention,Hazardous Energy Control MCP (Rev 1) – Briefin...,Security Awareness Refresher (1.0)
COURSE_GOOD,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
PERSON_GOOD,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
ID,343,158,351,76,27,398,165,257,406,212,220,131,169,267,261,222,272,234,156,255


In [25]:
if IN_COLAB:
    out_err.to_csv("out_errors.csv", index=False)
else:
    out_err.to_csv(genFileName(ERRORS_PRE, TIME_SUFF, FILE_EXT), index=False)

In [26]:
# OLD OUTPUT
#print(out_err.shape)
#out_err.to_csv("out_errors.csv", index=False)
#out_err.to_csv(genFileName(ERRORS_PRE, TIME_SUFF, FILE_EXT), index=False)

### Generate Lists of Missing Courses and People

Generate list of course IDs and usernames that are in the transcript data, but do not exist in Saba. We know these will fail since the requisite data does not exist in Saba. 

In [27]:
course_need = out_err[out_err["COURSE_GOOD"] == False]
person_need = out_err[out_err["PERSON_GOOD"] == False]

# Old Output
#pd.Series(course_need["COURSE"].unique(), name="Missing Courses").to_csv("out_courses.csv")
#pd.Series(person_need["LEARNER"].unique(), name="Missing Learners").to_csv("out_people.csv")
#pd.Series(course_need["COURSE"].unique(), name="Missing Courses").to_csv(genFileName(MISSING_COURSE, TIME_SUFF, FILE_EXT))
#pd.Series(person_need["LEARNER"].unique(), name="Missing Learners").to_csv(genFileName(MISSING_PEOPLE, TIME_SUFF, FILE_EXT))

In [28]:
if IN_COLAB:
    pd.Series(course_need["COURSE"].unique(), name="Missing Courses").to_csv("out_courses.csv")
    pd.Series(person_need["LEARNER"].unique(), name="Missing Learners").to_csv("out_people.csv")
else:
    pd.Series(course_need["COURSE"].unique(), name="Missing Courses").to_csv(genFileName(MISSING_COURSE, TIME_SUFF, FILE_EXT))
    pd.Series(person_need["LEARNER"].unique(), name="Missing Learners").to_csv(genFileName(MISSING_PEOPLE, TIME_SUFF, FILE_EXT))